In [36]:
import pandas as pd
import numpy as np
from path import Path  # pip install --user path.py
import re
from IPython.display import display
from pprint import pprint
import netCDF4
from IPython.core.debugger import Pdb
from collections import namedtuple

In [2]:
# load seaborn and other stuff for visualization
import seaborn  # pip install --user seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
def debug_here():
    Pdb().set_trace()

In [3]:
classes = pd.read_pickle("classes.pkl")
node_pairs = pd.read_pickle("paths_with_classes.pkl").drop("shortest_path", axis=1)

In [4]:
TEST_RESULT_DIRECTORIES = Path("/home/shibbiry/Dropbox/documents/msu/bachelors_thesis_cluster_topology/test_results") \
    .dirs()

In [5]:
def read_benchmark_hostnames(path_to_file):
    lines = path_to_file.lines()
    return (re.match(r"^(n\d{5})\.", line).groups()[0] for line in lines)

In [46]:
TestResults = namedtuple("TestResults", ["hostnames", "medians", "msg_lengths"])

In [125]:
def import_data(directory):
    hostnames = tuple(read_benchmark_hostnames(directory.joinpath("network_hosts.txt")))
    with netCDF4.Dataset(directory.joinpath("network_median.nc"), "r")  as dataset:
        step_len = dataset["step_length"][0]
        start_len = dataset["begin_mes_length"][0]
        end_len = dataset["end_mes_length"][0]
        
        assert len(hostnames) == dataset["proc_num"][0]
        assert dataset["test_type"][0] == 1
        assert start_len == 0
        assert end_len == 10000  # last message length should be 9900
        assert step_len == 100
        steps = (end_len - start_len) // step_len - 1
        assert start_len + (steps + 1) * step_len == end_len
        
        lengths = range(start_len, end_len, step_len)
        
        data = {
            "message_len_{0}".format(length): \
                pd.DataFrame(dataset["data"][index], index=hostnames, columns=hostnames)
            for (index, length) in enumerate(lengths)
        }
        panel = pd.Panel(data)
    return TestResults(hostnames=hostnames, medians=panel, msg_lengths=list(lengths))

In [124]:
test_results = import_data(TEST_RESULT_DIRECTORIES[0])

--Return--
None
> <ipython-input-8-795ae8f0e1b5>(2)debug_here()
      1 def debug_here():
----> 2     Pdb().set_trace()

ipdb> up
> <ipython-input-123-41e19adb6b2d>(4)import_data()
      2     hostnames = tuple(read_benchmark_hostnames(directory.joinpath("network_hosts.txt")))
      3     with netCDF4.Dataset(directory.joinpath("network_median.nc"), "r")  as dataset:
----> 4         debug_here()
      5         step_len = dataset["step_length"][0]
      6         start_len = dataset["begin_mes_length"][0]

ipdb> dataset
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    dimensions(sizes): x(100), y(100), n(100), strings(101)
    variables(dimensions): int32 proc_num(), int32 test_type(), int32 data_type(), int32 begin_mes_length(), int32 end_mes_length(), int32 step_length(), int32 noise_mes_length(), int32 num_noise_mes(), int32 num_noise_proc(), int32 num_repeates(), float64 data(n,x,y)
    groups: 

ipdb> a
directory = Path('/ho

## Make sure that all classes were covered by our test

In [48]:
def check_all_classes_covered(hostnames):
    """Fails an assertion if there is a class of pairs
    that was not covered by the test"""
    pairs_tested = node_pairs[
        node_pairs["node1"].isin(hostnames) &
        node_pairs["node2"].isin(hostnames)
    ]
    assert len(pairs_tested["class_"].unique()) == len(classes)

In [49]:
check_all_classes_covered(test_results.hostnames)

## More helper functions

In [75]:
class ClassGetter():
    def __init__(self, node_pairs):
        self.reverse_lookup_table = node_pairs.set_index(["node1", "node2"])
    
    def __call__(self, node1, node2):
        min_node = min(node1, node2)
        max_node = max(node1, node2)
        return self.reverse_lookup_table.loc[min_node].loc[max_node].loc["class_"]       

In [81]:
get_class = ClassGetter(node_pairs)

In [97]:
def get_values_from_class(matrix, class_):
    """returns an iterable with values from pairs
    that belong to a specific class"""
    hostnames = list(matrix.index)
    assert (matrix.columns == matrix.index).all()
    
    return (
        matrix.loc[node1].loc[node2] for node1 in hostnames
        for node2 in matrix.columns  
        if node1 != node2 and get_class(node1, node2) == class_  # dirty hack
    )

Whoopy doopy doops OOPS

these matrices have ping from node to self

I mean the diagonal is not empty, is it?

I mean (n48332, n48332) - is it always 0 or not? I must check!

In [103]:
pd.DataFrame({"A": [1, 2, 3], "b": [4, 5, 6]}).squeeze()

,A,b
0,1,4
1,2,5
2,3,6


In [128]:
uniques = list(frozenset(
    test_results.medians.iloc[99].loc[ind].loc[col]
    for col in test_results.medians.iloc[0].columns
    for ind in test_results.medians.iloc[0].index
))

In [100]:
list(get_values_from_class(test_results.medians.iloc[0], 5))

[4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,
 4.0531158447265625e-06,


## Prediction Algorithm

In [ ]:
def make_predictor(test_results):
    data = {
        {class_: ?? for class_ in classes.index}
        for length in test_results.msg_lengths
    }

In [ ]:
def predict(test_results, message_length, node1, node2):
    